In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import utility
import math
from time import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import classification_report

In [2]:
#SklearnDatasets
dataset = datasets.load_breast_cancer() # load_iris, load_wine, load_breast_cancer()
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
dir_path = 'Breast_Cancer'

In [3]:
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)

0.0 1.0000000000000002


In [4]:
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = dataset.target
df_scaled['target'] = targets
num_classes = len(df_scaled['target'].unique())
display(df_scaled)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,0.132056,...,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667,0
565,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,0.113100,...,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315,0
566,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,0.137321,...,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909,0
567,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,0.425442,...,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315,0


In [5]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
n_neurons = X_train.shape[1]
n_hidden_layers = 1
print(f'n_hidden_layers: {n_hidden_layers}, n_neurons {n_neurons}')
model = tf.keras.Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),  # Explicit input layer
    layers.Dense(n_neurons, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model_path = f'new_models/{dir_path}.keras'
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
ck = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
start = time()
model.fit(X_train,y_train, epochs=400, batch_size=32,validation_data=(X_test, y_test), verbose=1, callbacks=[ck, es])
print(f'Training time: {time()-start}')
model = tf.keras.models.load_model(model_path)
print('Training Results')
model.evaluate(X_train, y_train, verbose=2)
print('Test Results')
model.evaluate(X_test, y_test, verbose=2)
model.save(f'new_models/{dir_path}.h5', save_format='h5')

n_hidden_layers: 1, n_neurons 30


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 30)                  │             930 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │              62 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 992 (3.88 KB)

 Trainable params: 992 (3.88 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4175 - loss: 0.6944 - val_accuracy: 0.5691 - val_loss: 0.6659
Epoch 2/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5819 - loss: 0.6745 - val_accuracy: 0.6253 - val_loss: 0.6525
Epoch 3/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6535 - loss: 0.6613 - val_accuracy: 0.7002 - val_loss: 0.6381
Epoch 4/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6144 - loss: 0.6505 - val_accuracy: 0.7541 - val_loss: 0.6240
Epoch 5/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7426 - loss: 0.6331 - val_accuracy: 0.7892 - val_loss: 0.6100
Epoch 6/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7794 - loss: 0.6162 - val_accuracy: 0.8056 - val_loss: 0.5960
Epoch 7/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8285 - loss: 0.6046 - val_accuracy: 0.8361 - val_loss: 0.5806
Epoch 8/400
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8489 - loss: 0.5831 - val_accuracy: 0.8525 - val_

In [6]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

y_test_classes = np.argmax(y_test, axis=1)

report = classification_report(y_test_classes, y_pred_classes,digits=4)
print(report)

# Save the report to a text file
with open(f'new_models/{dir_path}_report.txt', 'w') as f:
    f.write(report)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0     0.9934    0.9434    0.9677       159
           1     0.9674    0.9963    0.9816       268

    accuracy                         0.9766       427
   macro avg     0.9804    0.9698    0.9747       427
weighted avg     0.9771    0.9766    0.9765       427

